In [1]:
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

import pandas as pd
from bs4 import BeautifulSoup as bs4
import numpy as np

import pickle

In [ ]:
df = pd.read_csv('spam_clean.csv', usecols=["spam", "clean"])
df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True) # removing NaNs

X_train, X_test, y_train, y_test = train_test_split(df['clean'], df['spam'], test_size=0.2)

# Bag of Words
bow = CountVectorizer()
X_train = bow.fit_transform(X_train)
X_test = bow.transform(X_test)

In [ ]:
# BoW
logreg = LogisticRegression(max_iter=1000, verbose=2)
logreg.fit(X_train, y_train)

In [ ]:
test_spam = 'Urgent! Please call 09061743810 from landline. Your ABTA complimentary 4* Tenerife Holiday or #5000 cash await collection SAE T&Cs Box 326 CW25WX 150 ppm'

def into_bow(text):
    stops = stopwords.words("english")
    porter = PorterStemmer()
    text = bs4(text).get_text()
    words = word_tokenize(text)
    words_no_punc = [word.lower() for word in words if word.isalpha()]
    no_stop = [word for word in words_no_punc if word not in stops]
    stems = [porter.stem(word) for word in no_stop]
    clean = ' '.join(stems)

    spam_bow = bow.transform([text])

    return spam_bow

spam = into_bow(test_spam)

# print(logreg.predict(spam))
# print(logreg.predict_proba(spam)[0,1])

# Pickeling

In [ ]:
pickle.dump(logreg, open('logreg.pkl', 'wb')) #Saving the model
pickle.dump(bow, open('bow.pkl', 'wb')) #Saving the model

In [2]:
test_spam = 'Urgent! Please call 09061743810 from landline. Your ABTA complimentary 4* Tenerife Holiday or #5000 cash await collection SAE T&Cs Box 326 CW25WX 150 ppm'

bow = pickle.load(open('bow.pkl', 'rb'))

def into_bow(text):
    stops = stopwords.words("english")
    porter = PorterStemmer()
    text = bs4(text).get_text()
    words = word_tokenize(text)
    words_no_punc = [word.lower() for word in words if word.isalpha()]
    no_stop = [word for word in words_no_punc if word not in stops]
    stems = [porter.stem(word) for word in no_stop]
    clean = ' '.join(stems)

    spam_bow = bow.transform([text])

    return spam_bow

spam = into_bow(test_spam)

In [3]:
model = pickle.load(open('logreg.pkl', 'rb'))

model.predict_proba(spam)

array([[0.0560143, 0.9439857]])